##### Requirements

In [ ]:
%pip install re
%pip install bs4
%pip install requests
%pip install pandas

##### Definição das funções de busca

Casos onde ou a empresa não respondeu, ou o consumidor não avaliou, tem uma estrutura html diferente das outras, com menos nós. Por isso o "sibling" pode ser None, e precisamos que de uma função auxiliar `TratarSibling` que retorna string vazia. 

Quando o nó existe, ele vem com alguns caracteres que atrapalham a construção do DataFrame, por isso a função já faz uma limpeza preliminar.

In [9]:
import re 
import bs4

# função auxiliar para tratar casos onde ou a empresa não respondeu, ou o consumidor não avaliou
def TratarSibling(sibling: bs4.element.Tag) -> str:
  """Trata Tags que podem ser vazias ou com sujeira no texto
  Arguments: 
    sibling: uma Tag que pode ser vazia (None) 
  Returns:
    string vazia se a Tag for vazia.
    Caso contrário, o texto da Tag sem quebras de linha."""
  if sibling is not None: 
    return re.sub('[\n\t\r\xa0\x0b]',' ',sibling.get_text()).strip()
  return ''

A quantidade de dias para resposta da empresa pode ser um texto no formato `(1 dia(s) depois)` ou `(no mesmo dia)`, ou ainda `<não respondido pela empresa>`. 

A função auxiliar `GetDiasDepois` se faz necessária para que transformemos esse texto em um número de dias

In [10]:
def GetDiasDepois(text: str):
  if text.endswith("mesmo dia)"):
    return 0
  if text.endswith("depois)"):
    return int(re.sub('[\n\t\(\)]|( dia\(s\) depois)','',text))
  return "N/A"

Definição da função `RetornarBuscaReclamacoes`, que realiza o web scraping das reclamações no site Consumidor.gov.br. 

Pela própria natureza do site, a busca retornará apenas 10 resultados, no máximo.

Além dos dados, a página retorna um índice para o próximo primeiro resultado. Se não houverem mais resultados, esse índice será -1.



In [11]:
import requests

# função para consultar, conforme os parametros passados, o site consumidor.gov.br
# e retornar as listas (tamanho máximo: 10):
# empresas, resolucoes, datas, cidades, estados, reclamacoes, respostas, diasResposta, notas e avaliacoes;
# e retornar o proximo indice que continua a busca
def RetornarBuscaReclamacoes(indicePrimeiroResultado='0', 
                             palavrasChave='', 
                             segmentoMercado='', 
                             fornecedor='', 
                             regiao='', 
                             uf='', 
                             cidade='',
                             area='', 
                             assunto='', 
                             problema='', 
                             dataInicio='', 
                             dataTermino='', 
                             avaliacao='', 
                             nota=''):
  """Consultar, conforme os parametros passados, o site consumidor.gov.br
  e retornar as listas (tamanho máximo: 10):
  empresas, resolucoes, datas, cidades, estados, reclamacoes, respostas, diasResposta, notas e avaliacoes;
  e retornar o proximo indice que continua a busca"""

  #definir parametros de busca
  data = {
    'indicePrimeiroResultado': indicePrimeiroResultado,
    'palavrasChave': palavrasChave,
    'segmentoMercado': segmentoMercado,
    'fornecedor': fornecedor,
    'regiao': regiao,
    'uf': uf,
    'cidade': cidade,
    'area': area,
    'assunto': assunto,
    'problema': problema,
    'dataInicio': dataInicio,
    'dataTermino': dataTermino,
    'avaliacao': avaliacao,
    'nota': nota,
  }

  response = requests.post(
      'https://consumidor.gov.br/pages/indicador/relatos/consultar',
      data=data,
  )

  # converter no objeto para busca dos elementos 
  page = bs4.BeautifulSoup(response.text, 'html.parser')
  
  # busca dos nomes das empresas
  aElems = page.select('a')
  empresas = [elem.get_text() for elem in aElems]

  # busca das informacoes Resolvido/Não Resolvido/Não avaliado pelo consumidor
  h4Elems = page.find_all('h4')
  resolucoes = [TratarSibling(elem) for elem in h4Elems]

  # busca das datas das reclamacoes do consumidor
  pElems = page.find_all(name="strong", string="Relato")
  datas = [TratarSibling(elem.findNextSibling())[0:10] for elem in pElems]  

  # busca das cidades e estado de origem do consumidor
  cidades_estados = [TratarSibling(elem.findNextSibling())[12:] for elem in pElems]
  cidades = [cidade_estado.split(' - ')[0] for cidade_estado in cidades_estados]
  estados = [cidade_estado.split(' - ')[1] for cidade_estado in cidades_estados]

  # busca do texto das reclamacoes do consumidor
  pElems = page.find_all('p', style="word-wrap: break-word;")
  reclamacoes = [TratarSibling(elem) for elem in pElems]
  
  # busca do texto das respostas das empresas
  pElems = page.find_all(name="strong", string="Resposta")
  respostas = [TratarSibling(elem.findNextSibling().findNextSibling()) for elem in pElems]
  
  # busca do numero de dias que as empresas levaram para responder
  diasResposta = [GetDiasDepois(TratarSibling(elem.findNextSibling())) for elem in pElems]

  # busca das notas dadas pelo consumidor
  pElems = page.find_all(name="strong", string="Avaliação")
  notas = [re.sub('[\n\t]|Nota|<.*?>','',TratarSibling(elem.findNextSibling())) for elem in pElems]

  # busca dos textos da avaliação dada pelo consumidor
  avaliacoes = [TratarSibling(elem.findNextSibling().findNextSibling()) for elem in pElems]

  # busca do proximo indice que ira trazer resultados. retorna -1 se acabou
  primeiroProximoIndice = int(re.search('-?\d+',page.find('script').text).group())

  return empresas, resolucoes, datas, cidades, estados, reclamacoes, respostas, diasResposta, notas, avaliacoes, primeiroProximoIndice

Como a função de busca retorna apenas 10 resultados, a função `RetornarBaseReclamacoes` ajuda a retornar todos os resultados da busca.

O custo computacional de aquisição dos dados medido nos testes foi de **1 segundo a cada 10 linhas buscadas**, quando o único parâmetro passado é a palavra chave. **Para mais parâmetros, o tempo de busca pode ser maior**.

Se desejar limitar o tempo de busca, basta passar o parâmetro `numero_maximo_resultados`.



In [12]:
import pandas as pd
import warnings

# função que retorna a base completa com todos os resultados da busca, conforme os parametros passados, ou uma base com numero_maximo_resultados
def RetornarBaseReclamacoes(
    numero_maximo_resultados=None, 
    palavrasChave=[''], 
    segmentoMercado='', 
    fornecedor='', 
    regiao='', 
    uf='', 
    cidade='',
    area='', 
    assunto='', 
    problema='', 
    dataInicio='', 
    dataTermino='', 
    avaliacao='', 
    nota='') -> pd.DataFrame:
  """Retorna a base completa com todos os resultados da busca, conforme os parametros passados, ou uma base com numero_maximo_resultados"""

  # aviso de parametro provavelmente sendo mal utilizado
  if numero_maximo_resultados == 0:
    warnings.warn("warning: se numero_maximo_resultados for 0, a busca inteira será retornada (pode demorar alguns minutos)")

  # se o valor de numero_maximo_resultados não for passado, definir um valor padrão para retornar tudo
  if numero_maximo_resultados is None:
    numero_maximo_resultados = 0

  # tratamento de parametro incorreto  
  if not type(numero_maximo_resultados) is int:
    raise TypeError("numero_maximo_resultados deve ser um int")
  
  if not type(palavrasChave) is list or not type(palavrasChave[0]) is str:
    raise TypeError("palavrasChave deve ser uma list de str")
  
  if numero_maximo_resultados < 0 or numero_maximo_resultados % 10 != 0:
    raise Exception("numero_maximo_resultados precisa ser um número inteiro positivo múltiplo de 10. Ex: 0, 10, 20, 1000...")  
  
  # criação do DataFrame sem dados. Os dados buscados serão concatenados neste
  df = pd.DataFrame(columns=['empresa', 'resolucao', 'data_reclamacao', 'cidade', 'estado','reclamacao', 'resposta', 'dias_depois_resposta', 'nota', 'avaliacao'])

  # para melhor desempenho na busca, vamos buscar uma palvra chave por vez
  for palavraChave in palavrasChave:

    # busca incial sempre começa com indice 0
    primeiroProximoIndice = 0

    # laço para realizar a busca até o último resultado ou até numero_maximo_resultados ser atingido
    while primeiroProximoIndice >= 0 and (numero_maximo_resultados == 0 or primeiroProximoIndice < numero_maximo_resultados):
      # buscas em sequencia utilizando sempre o indice retornado para buscar mais resultados
      print('buscando palavra chave "{0}" a partir do indice {1}...'.format(palavraChave, primeiroProximoIndice))
      empresas, resolucoes, datas, cidades, estados, reclamacoes, respostas, diasResposta, notas, avaliacoes, primeiroProximoIndice = RetornarBuscaReclamacoes(
          indicePrimeiroResultado=primeiroProximoIndice, 
          palavrasChave=palavraChave, 
          segmentoMercado=segmentoMercado, 
          fornecedor=fornecedor, 
          regiao=regiao, 
          uf=uf,
          cidade=cidade,
          area=area, 
          assunto=assunto, 
          problema=problema, 
          dataInicio=dataInicio, 
          dataTermino=dataTermino, 
          avaliacao=avaliacao, 
          nota=nota
          )
      df_aux = pd.DataFrame(list(zip(empresas, resolucoes, datas, cidades, estados, reclamacoes, respostas, diasResposta, notas, avaliacoes))
      , columns=['empresa', 'resolucao', 'data_reclamacao', 'cidade', 'estado', 'reclamacao', 'resposta', 'dias_depois_resposta', 'nota', 'avaliacao'])
      print('{0} resultados retornados'.format(len(df_aux)))
      df = pd.concat([df, df_aux], ignore_index=True)
      print('{0} resultados até o momento'.format(len(df)))

  return df

##### Definição dos parametros de busca

In [13]:
palavrasChave=['atendimento']
numero_maximo_resultados = 10 # Se 0 ou None: retornar tudo.
segmentoMercado=''
fornecedor=''
regiao=''
uf=''
cidade=''
area=''
assunto=''
problema=''
dataInicio='' 
dataTermino=''
avaliacao=''
nota=''

##### Aquisição dos dados e montagem da base

In [14]:
df = RetornarBaseReclamacoes(numero_maximo_resultados,
                             palavrasChave,
                             segmentoMercado,
                             fornecedor,
                             regiao,
                             uf,
                             cidade,
                             area,
                             assunto,
                             problema,
                             dataInicio,
                             dataTermino,
                             avaliacao,
                             nota)

buscando palavra chave "atendimento" a partir do indice 0...
10 resultados retornados
10 resultados até o momento


Exibir resultado

In [15]:
display(df)

,empresa,resolucao,data_reclamacao,cidade,estado,reclamacao,resposta,dias_depois_resposta,nota,avaliacao
0,Airbnb,Não Resolvido,"09/12/2022,",Taubaté,SP,Entre os dia 03 e 05 de dezembro de 2022 eu me...,"Caro consumidor, Obrigado por entrar em con...",0,1,O Airbnb se negou a efetuar o reembolso pelos ...
1,Latam Airlines (Tam),Resolvido,"09/12/2022,",Brasília,DF,Realizei a compra da passagem BSB-JDO-BSB em 6...,"Prezado(a) Cliente, Informamos que providen...",0,4,Não ofereceram reembolso (como determina o art...
2,Drogaria São Paulo,Resolvido,"08/12/2022,",São Paulo,SP,Fiz uma compra no dia 04 de Dezembro pedido nº...,"Olá, João, boa tarde! Inicialmente peço des...",0,2,sim...os produtos foram entregues hoje um pouc...
3,Cartão Cencosud GBarbosa,Resolvido,"08/12/2022,",Lauro de Freitas,BA,"Tentei contato na Central de atendimento, mas ...","Olá Diana, tudo bem? Acabamos de responder a...",0,5,A empresa apresentou proposta e aceitei
4,99App,Não Resolvido,"08/12/2022,",Curitiba,PR,Inclui o mesmo endereço como origem e destino ...,"Prezado solicitante, Em resposta à manifest...",1,1,Se restringiram a justificar a variação de pre...
5,Pernambucanas Cartões,Resolvido,"08/12/2022,",Belo Horizonte,MG,Liguei para a central de atendimento da Pernam...,"Olá Samantha, Recebemos a sua manifestação...",1,5,<não há comentários do consumidor>
6,Rappi,Resolvido,"08/12/2022,",Brasília,DF,"Tive uma série de problemas com a empresa, que...","Prezada, Lamento que a senhora tenha passa...",0,5,Agora resolveram.
7,Samsung,Não Resolvido,"08/12/2022,",Santo André,SP,Olá. Há mais ou menos dois meses o celular ...,"Caro Cliente, Informamos que os esclarecime...",1,1,Reabrir este ticket imediatamente pois o retor...
8,Latam Airlines (Tam),Resolvido,"08/12/2022,",Paracatu,MG,No dia 06/12/2022 entrei em contato com a Lata...,"Prezado(a) Cliente, Informamos que providen...",0,2,O atendimento do Carlos Lopes foi nota 10.
9,MaxMilhas,Resolvido,"08/12/2022,",Mossoró,RN,Após colocar a venda milhas smiles na max milh...,"Olá Clélia, A compra foi realizada pelo pass...",1,5,Apesar de não conseguir atendimento por ligaçã...


Salvar resultado em arquivos

In [17]:
df.to_csv(f"ReclamacoesConsumidor-{len(df)}-{'_'.join(palavrasChave)}.csv")
df.to_excel(f"ReclamacoesConsumidor-{len(df)}-{'_'.join(palavrasChave)}.xlsx")